- Inicializou a população ;
- Quer calcular o fitness dos pais :
  - Indice ibovespa;
  - Media ponderada dos indices de crescimento de cada cromossomo:
    - Pra cada alelo:
      - (Close - Open)/Open ; __só para um dia??__
  - Fitness = min((Média - Indice)^2)
  

- Indice no dia;

-Dataframe =  2 colunas:
 - Primeira: Label;
 - Segunda: Crescimento no dia;
 - Um dia em particular.

In [6]:
import pandas as pd

ibvday = pd.read_excel("dados/IBOVDIA.XLS", header = 1)

In [9]:
ibvday.set_index("PREGÃO", inplace = True)

In [17]:
ibvday.head()

,JAN,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
PREGÃO,,,,,,,,,,,,
1,NaN,NaN,NaN,9138.3,NaN,NaN,13002.2,12259.1,10109.1,12173.4,NaN,9648.4
2,6955.54,NaN,NaN,9237.3,10074.2,11360.8,13259.6,NaN,11062.9,12396.6,NaN,9964.3
3,6997.75,8095.94,8978.2,9302.1,NaN,11110.4,13284.8,NaN,11119.4,12540.9,9858.0,9991.7
4,NaN,8102.25,8961.4,9705.2,NaN,11029.9,13493.8,12016.9,11430.6,NaN,10254.1,10046.3
5,NaN,8171.18,9074.4,NaN,9993.5,11106.3,NaN,11908.6,11838.3,NaN,9988.3,10080.0


In [31]:
ibvday.loc[9]["JUN"]

11396.0

In [26]:
luanindex = ((ibvday.iloc[9]["JUN"] - ibvday.iloc[8]["JUN"])/ibvday.iloc[8]["JUN"]) 

In [32]:
import bovespa as bv
import deap as dp

In [34]:
empresas_top = ["CSNA3", "RADL3", "BVMF3", "MULT3",
                "SBSP3", "CPFE3", "BBDC4", "BRML3",
                "BBDC3", "EQTL3", "CSAN3", "PETR4",
                "SMLE3", "LREN3", "KROT3", "PETR3",
                "USIM5", "CMIG4", "QUAL3", "MRVE3",
                "GGBR4", "VALE5", "CCRO3", "VIVT4",
                "VALE3", "GOAU4", "ITSA4", "CIEL3",
                "TBLE3", "EMBR3"]

In [ ]:
for rec in bf.query():
    if rec.year == 2018 and (rec.month == 6) and (rec.day == 9 or rec.day == 10):
        acc.append(rec)

In [58]:
bf = bv.File("dados/COTAHIST_A2018.TXT")
acc = []
for rec in bf.query():
    if rec.date.year == 2018 and (rec.date.month == 6) :
        acc.append(rec)

In [64]:
foo = acc[0]

In [90]:
date = pd.Series(map(lambda foo: foo.date, acc))
price_open = pd.Series(map(lambda foo: foo.price_open, acc))
price_close = pd.Series(map(lambda foo : foo.price_close, acc))
stock_code = pd.Series(map(lambda foo: foo.stock_code, acc))
company_name = pd.Series(map(lambda foo : foo.company_name, acc))

In [91]:
df = pd.concat([date,price_open, price_close, company_name, stock_code],
               axis=1)
df.columns = ['date', 'price_open', "price_close", "company_name", "stock_code"]

In [92]:
topdf = df[df['stock_code'].isin(empresas_top)]

In [93]:
topdf

,date,price_open,price_close,company_name,stock_code
206,2018-06-01,26.41,27.10,BRADESCO,BBDC3
208,2018-06-01,29.60,29.88,BRADESCO,BBDC4
427,2018-06-01,10.30,10.35,BR MALLS PAR,BRML3
458,2018-06-01,10.31,10.30,CCR SA,CCRO3
488,2018-06-01,16.90,17.29,CIELO,CIEL3
515,2018-06-01,7.70,7.60,CEMIG,CMIG4
540,2018-06-01,23.00,23.61,CPFL ENERGIA,CPFE3
562,2018-06-01,39.50,38.45,COSAN,CSAN3
572,2018-06-01,7.81,7.77,SID NACIONAL,CSNA3
674,2018-06-01,22.67,22.79,EMBRAER,EMBR3


In [87]:
df["company_name"].unique()

array(['ALLIAR', 'APPLE', 'ABC BRASIL', 'FII ABC IMOB', 'AMBEV S/A',
       'ABEVE', 'ABEV FM', 'ABEV', 'ABEVE FM', 'ABBOTT', 'ADVANCED-DH',
       'FII AESAPAR', 'FII AG CAIXA', 'BRASILAGRO', 'FII TORRE AL',
       'ALPARGATAS', 'ALIANSCE', 'ALUPAR', 'FII ALIANZA', 'LOJAS MARISA',
       'AMAZON', 'ANIMA', 'ARCELOR', 'AREZZO CO', 'ATOMPAR',
       'FII CSHGATSA', 'ATT INC', 'AZUL', 'B3', 'B3SAE', 'B3SA FM',
       'B3SA', 'B3SAE FM', 'BAHEMA', 'EXCELSIOR', 'AMAZONIA', 'BRASIL',
       'BBAS', 'BBASE /EJ', 'BBAS /EJ', 'BBAS FM/EJ', 'BBASE',
       'BBASE FM/EJ', 'BRADESCO', 'BBDCE', 'BBDC FM', 'BBDC', 'BBDCE FM',
       'FII BB PROGR', 'FII BB PRGII', 'FII BB CORP', 'BR BROKERS',
       'BBSEGURIDADE', 'BBSE', 'BBSE FM', 'BBSEE', 'BBSEE FM',
       'FII C JARDIM', 'FII BC FFII', 'FII BCIA', 'FII BEES CRI',
       'MINERVA', 'BANESTES', 'BERKSHIRE', 'BANESE', 'INTER BANCO',
       'BIOMM', 'BK BRASIL', 'MERC BRASIL', 'MERC INVEST', 'BIC MONARK',
       'FII BMBRC LC', 'NORD BRASIL', 'FI

In [2]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [3]:
import random
from deap import tools

IND_SIZE = 12

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

/home/marcelovmaciel/miniconda3/lib/python3.7/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [4]:
toolbox

In [5]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [8]:
from tqdm import tqdm
    
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 200

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in tqdm(range(NGEN)):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [9]:
main()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 715.29it/s]


[[-14.769356900626843,
  -22.42133995343212,
  -14.348365063303605,
  -17.969991578358957,
  -14.292749073391796,
  -18.459667845059617,
  -25.054935607636132,
  -16.85185114098054,
  -17.712898727535006,
  -21.635660153637154,
  -15.240201994956927,
  -15.366488039067333],
 [-14.769356900626843,
  -22.42133995343212,
  -14.348365063303605,
  -17.969991578358957,
  -14.292749073391796,
  -18.459667845059617,
  -25.054935607636132,
  -16.85185114098054,
  -17.712898727535006,
  -20.6121984590088,
  -15.240201994956927,
  -15.366488039067333],
 [-14.769356900626843,
  -24.341706995859248,
  -14.348365063303605,
  -17.428848948081445,
  -13.658240533607422,
  -19.70997141103366,
  -27.187290417695525,
  -16.85185114098054,
  -17.712898727535006,
  -21.635660153637154,
  -15.240201994956927,
  -15.366488039067333],
 [-14.720111842441852,
  -24.341706995859248,
  -14.348365063303605,
  -17.428848948081445,
  -13.462424926310517,
  -19.70997141103366,
  -25.6101504755255,
  -16.8518511409805